**Library**

In [1]:
import sys
import os
import yaml
import json
import pandas as pd
import random
from sklearn_crfsuite import metrics
from sklearn.metrics import classification_report, accuracy_score

In [2]:
sys.path.append(os.path.abspath(os.path.join('..')))
from src.features.hand_crafted import sent2features, sent2labels, sent2tokens, get_relation_features
from src.models.machine_learning import CRFModel, FlatModelWrapper, RelationExtractionModel
from src.data_loader.dataset import convert_label_studio_to_ner_data, prepare_re_data_from_json
from sklearn.ensemble import RandomForestClassifier

**Load Config**

In [3]:
with open('../configs/ml_config.yaml', 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)
print("Load config thành công!")

Load config thành công!


**Load Data**

In [4]:
print("Loading data files...")
TRAIN_PATH = '../data/04_model_input/train_dataset.json'
TEST_PATH = '../data/04_model_input/test_dataset.json'
DEV_PATH = '../data/04_model_input/dev_dataset.json'

with open(TRAIN_PATH, 'r', encoding='utf-8') as f:
    train_json = json.load(f)

with open(TEST_PATH, 'r', encoding='utf-8') as f:
    test_json = json.load(f)

with open(DEV_PATH, 'r', encoding='utf-8') as f:
    dev_json = json.load(f)

print(f"-> Train raw docs: {len(train_json)}")
print(f"-> Test raw docs: {len(test_json)}")
print(f"-> Dev raw docs: {len(dev_json)}")

Loading data files...
-> Train raw docs: 797
-> Test raw docs: 114
-> Dev raw docs: 228


In [5]:
print("\nConverting to BIO format...")
train_sents = convert_label_studio_to_ner_data(train_json)
test_sents = convert_label_studio_to_ner_data(test_json)
dev_sents = convert_label_studio_to_ner_data(dev_json)


Converting to BIO format...


Converting with Pyvi: 100%|██████████| 228/228 [00:00<00:00, 775.77it/s]


In [6]:
train_sents[0][:20]

[('Bán', 'O'),
 ('lô', 'B-TYPE'),
 ('đất', 'I-TYPE'),
 ('100m²', 'B-AREA'),
 ('ngay', 'O'),
 ('cạnh', 'O'),
 ('nhà', 'O'),
 ('thi_đấu', 'B-ATTR'),
 ('Thủ_Thừa', 'I-ATTR'),
 (',', 'O'),
 ('view', 'B-ATTR'),
 ('hồ', 'I-ATTR'),
 ('sinh_thái', 'I-ATTR'),
 ('mát_mẻ', 'I-ATTR'),
 (',', 'O'),
 ('giá', 'O'),
 ('mềm', 'O'),
 (',', 'O'),
 ('xây_dựng', 'O'),
 ('ở', 'O')]

**Extracting features**

In [7]:
print("\nExtracting features")
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

print(f"-> X_train shape: {len(X_train)} sentences")
print(f"-> X_test shape: {len(X_test)} sentences")


Extracting features
-> X_train shape: 797 sentences
-> X_test shape: 114 sentences


In [8]:
def evaluate_model(y_true, y_pred, model_name):
    print(f"\n{'='*20} EVALUATION: {model_name} {'='*20}")
    
    # Lấy danh sách nhãn thực tế (trừ O để F1-score phản ánh đúng thực thể)
    labels = list(set([l for sent in y_true for l in sent]))
    if 'O' in labels: labels.remove('O')
    labels = sorted(labels)
    
    print(f"Accuracy: {metrics.flat_accuracy_score(y_true, y_pred):.4f}")
    print(metrics.flat_classification_report(
        y_true, y_pred, labels=labels, digits=4
    ))

### **NER**

**CRF (Conditional Random Fields)**

In [9]:
print("Training CRF model")
crf = CRFModel(config.get('crf', {}))
crf.train(X_train, y_train)

Training CRF model


In [10]:
# Predict & Evaluate
y_pred_crf = crf.predict(X_test)
evaluate_model(y_test, y_pred_crf, "CRF")


==================== EVALUATION: CRF ====================
Accuracy: 0.7206
              precision    recall  f1-score   support

      B-AREA     0.6199    0.6136    0.6167       295
      B-ATTR     0.5854    0.4800    0.5275       900
       B-LOC     0.6005    0.6501    0.6243       363
         B-O     0.1515    0.1250    0.1370        40
       B-ORG     0.7500    0.1579    0.2609        38
       B-PER     0.4348    0.2703    0.3333        37
     B-PRICE     0.7803    0.8438    0.8108       160
      B-TYPE     0.6395    0.5528    0.5930       199
      I-AREA     0.7143    0.6229    0.6655       297
      I-ATTR     0.5596    0.5108    0.5341      1296
       I-LOC     0.5957    0.6336    0.6141       393
         I-O     0.1515    0.0806    0.1053        62
       I-ORG     0.7619    0.6957    0.7273        23
       I-PER     0.3750    0.2222    0.2791        27
     I-PRICE     0.7188    0.8789    0.7909       256
      I-TYPE     0.3734    0.3933    0.3831       150

   m

In [11]:
# OUTPUT_MODEL_DIR = '../outputs/models' 
# os.makedirs(OUTPUT_MODEL_DIR, exist_ok=True)

# # Lưu model
# crf.save(os.path.join(OUTPUT_MODEL_DIR, 'crf_model.pkl'))

**SVM (Support Vector Machine)**

In [12]:
print("Training SVM Model")
svm = FlatModelWrapper('svm', config.get('svm', {}))
svm.train(X_train, y_train)

Training SVM Model


In [13]:
# Predict & Evaluate
y_pred_svm = svm.predict(X_test)
evaluate_model(y_test, y_pred_svm, "SVM")


==================== EVALUATION: SVM ====================
Accuracy: 0.7439
              precision    recall  f1-score   support

      B-AREA     0.6278    0.6576    0.6424       295
      B-ATTR     0.5922    0.5422    0.5661       900
       B-LOC     0.6677    0.6198    0.6429       363
         B-O     0.2000    0.1000    0.1333        40
       B-ORG     0.5714    0.1053    0.1778        38
       B-PER     0.4167    0.1351    0.2041        37
     B-PRICE     0.8070    0.8625    0.8338       160
      B-TYPE     0.6465    0.6432    0.6448       199
      I-AREA     0.6704    0.6094    0.6384       297
      I-ATTR     0.6077    0.5355    0.5693      1296
       I-LOC     0.6061    0.6616    0.6326       393
         I-O     0.1176    0.0323    0.0506        62
       I-ORG     0.5714    0.1739    0.2667        23
       I-PER     0.5000    0.3333    0.4000        27
     I-PRICE     0.7419    0.8984    0.8127       256
      I-TYPE     0.4819    0.5333    0.5063       150

   m

In [14]:
# OUTPUT_MODEL_DIR = '../outputs/models' 
# os.makedirs(OUTPUT_MODEL_DIR, exist_ok=True)

# # Lưu model
# svm.save(os.path.join(OUTPUT_MODEL_DIR, 'svm_model.pkl'))

**MaxEnt (Maximum Entropy)**

In [15]:
print("Training MaxEnt Model")
maxent = FlatModelWrapper('maxent', config.get('maxent', {}))
maxent.train(X_train, y_train)

Training MaxEnt Model


C:\Users\LEGION\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [16]:
y_pred_maxent = maxent.predict(X_test)
evaluate_model(y_test, y_pred_maxent, "MaxEnt")


==================== EVALUATION: MaxEnt ====================
Accuracy: 0.7291
              precision    recall  f1-score   support

      B-AREA     0.6242    0.6475    0.6356       295
      B-ATTR     0.6197    0.4744    0.5374       900
       B-LOC     0.5878    0.6639    0.6235       363
         B-O     0.1143    0.1000    0.1067        40
       B-ORG     0.6000    0.0789    0.1395        38
       B-PER     0.5000    0.3514    0.4127        37
     B-PRICE     0.7500    0.8438    0.7941       160
      B-TYPE     0.6519    0.5930    0.6211       199
      I-AREA     0.7160    0.6027    0.6545       297
      I-ATTR     0.6024    0.5085    0.5515      1296
       I-LOC     0.6081    0.6081    0.6081       393
         I-O     0.2273    0.0806    0.1190        62
       I-ORG     0.6667    0.2609    0.3750        23
       I-PER     0.4375    0.2593    0.3256        27
     I-PRICE     0.7324    0.8555    0.7892       256
      I-TYPE     0.4313    0.4600    0.4452       150

 

In [17]:
# OUTPUT_MODEL_DIR = '../outputs/models' 
# os.makedirs(OUTPUT_MODEL_DIR, exist_ok=True)

# # Lưu model
# maxent.save(os.path.join(OUTPUT_MODEL_DIR, 'maxent_model.pkl'))

In [18]:
def visualize_ner_predictions(test_sents, y_true, y_pred, num_samples=2):
    print(f"\n{'='*20} NER PREDICTION SAMPLES {'='*20}")
    
    # Chọn ngẫu nhiên một vài câu để hiển thị
    indices = random.sample(range(len(test_sents)), min(num_samples, len(test_sents)))
    count = 0
    for idx in indices:
        print(f"\n📌 Sentence #{idx}:")
        tokens = [t[0] for t in test_sents[idx]] 
        true_labels = y_true[idx]
        pred_labels = y_pred[idx]

        results = ['✅' if t == p else '❌' for t, p in zip(true_labels, pred_labels)]
        count += sum(1 for r in results if r == '✅')
        print(f"-> Correct Predictions in this sentence: {sum(1 for r in results if r == '✅')} out of {len(results)} tokens.")
        # Tạo DataFrame
        df = pd.DataFrame({
            'Token': tokens,
            'Thực tế (True)': true_labels,
            'Dự đoán (Pred)': pred_labels,
            'Kết quả': results
        })
        display(df)

In [19]:
print("\nVisualizing NER Predictions for CRF Model")
visualize_ner_predictions(test_sents, y_test, y_pred_crf)


Visualizing NER Predictions for CRF Model

==================== NER PREDICTION SAMPLES ====================

📌 Sentence #104:
-> Correct Predictions in this sentence: 29 out of 37 tokens.


,Token,Thực tế (True),Dự đoán (Pred),Kết quả
0,Cần,O,O,✅
1,bán,O,O,✅
2,2,O,O,✅
3,nền,B-TYPE,B-TYPE,✅
4,liền,O,I-TYPE,❌
5,kề,O,I-TYPE,❌
6,đường,B-LOC,B-LOC,✅
7,số,I-LOC,I-LOC,✅
8,1,I-LOC,I-LOC,✅
9,khu,B-LOC,B-LOC,✅



📌 Sentence #103:
-> Correct Predictions in this sentence: 47 out of 63 tokens.


,Token,Thực tế (True),Dự đoán (Pred),Kết quả
0,Chính,B-PER,B-PER,✅
1,chủ,I-PER,I-PER,✅
2,gửi,O,O,✅
3,bán,O,O,✅
4,2,O,O,✅
...,...,...,...,...
58,1,B-PRICE,B-PRICE,✅
59,tỷ,I-PRICE,I-PRICE,✅
60,750,I-PRICE,I-PRICE,✅
61,triệu,I-PRICE,I-PRICE,✅


In [20]:
print("\nVisualizing NER Predictions for SVM Model")
visualize_ner_predictions(test_sents, y_test, y_pred_svm)


Visualizing NER Predictions for SVM Model

==================== NER PREDICTION SAMPLES ====================

📌 Sentence #106:
-> Correct Predictions in this sentence: 30 out of 42 tokens.


,Token,Thực tế (True),Dự đoán (Pred),Kết quả
0,Giá,O,O,✅
1,500,B-PRICE,B-PRICE,✅
2,triệu,I-PRICE,I-PRICE,✅
3,sở_hữu,O,O,✅
4,ngay,O,O,✅
5,lô,B-TYPE,B-TYPE,✅
6,đất,I-TYPE,I-TYPE,✅
7,có,O,O,✅
8,đầy_đủ,B-ATTR,I-ATTR,❌
9,tiện_ích,I-ATTR,O,❌



📌 Sentence #3:
-> Correct Predictions in this sentence: 275 out of 328 tokens.


,Token,Thực tế (True),Dự đoán (Pred),Kết quả
0,Chỉ,O,O,✅
1,sau,O,O,✅
2,có,O,O,✅
3,1,O,O,✅
4,tháng,O,O,✅
...,...,...,...,...
323,nhận,O,O,✅
324,bàn_giao,O,O,✅
325,nhà,O,O,✅
326,về,O,O,✅


In [21]:
print("\nVisualizing NER Predictions for Maxent Model")
visualize_ner_predictions(test_sents, y_test, y_pred_maxent)


Visualizing NER Predictions for Maxent Model

==================== NER PREDICTION SAMPLES ====================

📌 Sentence #107:
-> Correct Predictions in this sentence: 70 out of 96 tokens.


,Token,Thực tế (True),Dự đoán (Pred),Kết quả
0,Chính,B-ATTR,B-PER,❌
1,chủ,I-ATTR,I-PER,❌
2,cần,O,O,✅
3,bán,O,O,✅
4,gấp,O,O,✅
...,...,...,...,...
91,tại,O,O,✅
92,phòng,O,O,✅
93,công_chứng,O,I-ATTR,❌
94,",",O,O,✅



📌 Sentence #54:
-> Correct Predictions in this sentence: 31 out of 42 tokens.


,Token,Thực tế (True),Dự đoán (Pred),Kết quả
0,Shophouse,B-TYPE,B-TYPE,✅
1,4,B-ATTR,O,❌
2,tầng,I-ATTR,O,❌
3,giá,O,O,✅
4,tốt,O,O,✅
5,nhất,O,O,✅
6,khu,O,B-LOC,❌
7,DT,B-AREA,I-LOC,❌
8,6x20,I-AREA,B-AREA,❌
9,Xây_dựng,O,O,✅


### **RE**

In [22]:
print("Creating pairs from Training Data...")
# Lưu ý: train_json, test_json đã được load ở phần NER bên trên
train_pairs = prepare_re_data_from_json(train_json)
test_pairs = prepare_re_data_from_json(test_json)

print(f"-> Total Train Pairs: {len(train_pairs)}")
print(f"-> Total Test Pairs: {len(test_pairs)}")

# 3. Trích xuất đặc trưng & Vector hóa
print("\nExtracting features for Pairs...")

Creating pairs from Training Data...
-> Total Train Pairs: 277036
-> Total Test Pairs: 47950

Extracting features for Pairs...


In [23]:
def extract_re_features(pairs):
    X = []
    y = []
    for p in pairs:
        feats = get_relation_features(p['text'], p['ent1'], p['ent2'])
        X.append(feats)
        y.append(p['label'])
    return X, y

In [24]:
X_train_re, y_train_re = extract_re_features(train_pairs)
X_test_re, y_test_re = extract_re_features(test_pairs)

In [25]:
# from sklearn.feature_extraction import DictVectorizer

In [26]:
# re_vectorizer = DictVectorizer(sparse=True)
# X_train_re = re_vectorizer.fit_transform(X_train_re_dict)
# X_test_re = re_vectorizer.transform(X_test_re_dict)

# print(f"RE Feature Matrix Shape: {X_train_re.shape}")

In [27]:
print("Training SVM for RE")
svm = RelationExtractionModel('svm', config.get('svm', {}))
svm.train(X_train_re, y_train_re)

Training SVM for RE


In [28]:
print("\n--- SVM Evaluation ---")
y_pred_svm = svm.predict(X_test_re)
acc = accuracy_score(y_test_re, y_pred_svm)

labels_re = list(svm.model.classes_)
if 'NO_RELATION' in labels_re:
    labels_re.remove('NO_RELATION')

print(f"Accuracy: {acc:.4f} ({acc*100:.2f}%)")
print("-" * 30)
print("Relation Extraction Performance:")
print(classification_report(y_test_re, y_pred_svm, labels=labels_re))


--- SVM Evaluation ---
Accuracy: 0.9696 (96.96%)
------------------------------
Relation Extraction Performance:
              precision    recall  f1-score   support

    HAS_AREA       0.68      0.64      0.66       273
    HAS_ATTR       0.45      0.95      0.61       919
   HAS_PRICE       0.49      0.26      0.34       146
  LOCATED_AT       0.74      0.62      0.67       297

   micro avg       0.51      0.78      0.61      1635
   macro avg       0.59      0.62      0.57      1635
weighted avg       0.55      0.78      0.61      1635



In [29]:
print("Training Maxent for RE")
maxent = RelationExtractionModel('maxent', config.get('maxent', {}))
maxent.train(X_train_re, y_train_re)

Training Maxent for RE


C:\Users\LEGION\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [30]:
print("\n--- Maxent Evaluation ---")
y_pred_maxent = maxent.predict(X_test_re)
acc = accuracy_score(y_test_re, y_pred_maxent)

labels_re = list(maxent.model.classes_)
if 'NO_RELATION' in labels_re:
    labels_re.remove('NO_RELATION')

print(f"Accuracy: {acc:.4f} ({acc*100:.2f}%)")
print("-" * 30)
print("Relation Extraction Performance:")
print(classification_report(y_test_re, y_pred_maxent, labels=labels_re))


--- Maxent Evaluation ---
Accuracy: 0.9803 (98.03%)
------------------------------
Relation Extraction Performance:
              precision    recall  f1-score   support

    HAS_AREA       0.67      0.60      0.64       273
    HAS_ATTR       0.70      0.82      0.76       919
   HAS_PRICE       0.54      0.51      0.52       146
  LOCATED_AT       0.71      0.80      0.75       297

   micro avg       0.69      0.75      0.72      1635
   macro avg       0.66      0.68      0.67      1635
weighted avg       0.68      0.75      0.71      1635



In [31]:
print("\nTraining Random Forest for RE")
rf = RelationExtractionModel('random_forest', config.get('random_forest', {}))
rf.train(X_train_re, y_train_re)


Training Random Forest for RE


In [32]:
print("\n--- Random Forest Evaluation ---")
y_pred_rf = rf.predict(X_test_re)
acc = accuracy_score(y_test_re, y_pred_rf)

labels_re = list(rf.model.classes_)
if 'NO_RELATION' in labels_re:
    labels_re.remove('NO_RELATION')

print(f"Accuracy: {acc:.4f} ({acc*100:.2f}%)")
print("-" * 30)
print("Relation Extraction Performance:")
print(classification_report(y_test_re, y_pred_rf, labels=labels_re))


--- Random Forest Evaluation ---
Accuracy: 0.9790 (97.90%)
------------------------------
Relation Extraction Performance:
              precision    recall  f1-score   support

    HAS_AREA       0.64      0.63      0.63       273
    HAS_ATTR       0.68      0.76      0.72       919
   HAS_PRICE       0.55      0.56      0.55       146
  LOCATED_AT       0.74      0.77      0.76       297

   micro avg       0.67      0.72      0.70      1635
   macro avg       0.65      0.68      0.66      1635
weighted avg       0.67      0.72      0.69      1635



In [37]:
def visualize_re_model_comparison(test_pairs, y_true, preds_dict, num_samples=10):
    print(f"\n{'='*30} RE MODEL COMPARISON {'='*30}")
    
    # Random index
    indices = random.sample(range(len(test_pairs)), min(num_samples, len(test_pairs)))
    
    data = []
    for idx in indices:
        pair = test_pairs[idx]
        # Format cặp thực thể
        ent1 = pair['ent1']['text']
        ent2 = pair['ent2']['text']
        pair_str = f"{ent1} ➡ {ent2}"
        true_label = y_true[idx]
        
        row = {
            'Cặp thực thể': pair_str,
            'Thực tế (True)': true_label
        }
        
        for model_name, y_pred in preds_dict.items():
            pred = y_pred[idx]
            status = "✅" if pred == true_label else "❌"
            row[f"{model_name}"] = f"{pred} {status}"
            
        data.append(row)
    
    df = pd.DataFrame(data)
    
    # Hiển thị
    display(df)

In [39]:
preds_dict = {
    'SVM': y_pred_svm,
    'MaxEnt': y_pred_maxent,
    'RandomForest': y_pred_rf
}

visualize_re_model_comparison(test_pairs, y_test_re, preds_dict, num_samples=15)


============================== RE MODEL COMPARISON ==============================


,Cặp thực thể,Thực tế (True),SVM,MaxEnt,RandomForest
0,"Mã căn S0715 ➡ 94,53m²",NO_RELATION,NO_RELATION ✅,NO_RELATION ✅,NO_RELATION ✅
1,"95,39m² ➡ 94,97m²",NO_RELATION,NO_RELATION ✅,NO_RELATION ✅,NO_RELATION ✅
2,"300 căn ➡ 19,5 tỷ",NO_RELATION,NO_RELATION ✅,NO_RELATION ✅,NO_RELATION ✅
3,"Liền kề Biệt thự ➡ 19,5 tỷ",NO_RELATION,NO_RELATION ✅,HAS_PRICE ❌,HAS_PRICE ❌
4,đất sổ đỏ ➡ lô góc,NO_RELATION,NO_RELATION ✅,NO_RELATION ✅,NO_RELATION ✅
5,kết nối đường Láng Lớn Kim Long ➡ 30 tỷ,NO_RELATION,NO_RELATION ✅,NO_RELATION ✅,NO_RELATION ✅
6,"Shophouse đại lộ 51m ➡ 8,x 9,x tỷ",NO_RELATION,NO_RELATION ✅,NO_RELATION ✅,NO_RELATION ✅
7,trục đường ven biển đẹp nhất Bắc Nha Trang ➡ g...,NO_RELATION,NO_RELATION ✅,NO_RELATION ✅,NO_RELATION ✅
8,"diện tích 1514,7 m² ➡ Khánh Vĩnh",NO_RELATION,NO_RELATION ✅,NO_RELATION ✅,NO_RELATION ✅
9,Huyện Bến Cầu ➡ xây trọ phục vụ công nhân,NO_RELATION,NO_RELATION ✅,NO_RELATION ✅,NO_RELATION ✅
